**IMPORTS**

In [1]:
from aux_functions import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


**DATA PROCESSING**

- Importing the pgn data
- Transforming the data to sparce tensors 
- Splitting the data into training and testing

In [2]:
TEST_PERCENT = 0.25

# Load pgn paths
pgns = import_data(1)

# Convert pgns to tensors
board_tensors, next_moves = parse_pgn_to_tensors(pgns)

# Converting the dataset into a custom pytorch one
dataset = ChessDataset(board_tensors, next_moves)

# Splitting the data into train and test
train_data, test_data = torch.utils.data.random_split(dataset, [1-TEST_PERCENT, TEST_PERCENT])

print(len(test_data))  # Number of states

**NEURAL NETWORK DESIGN**

In [4]:
BATCH_SIZE = 32

# Whether to do the operations on the cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PieceToMoveNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Takes as input a tensor of 14 channels (8x8 board)
        self.conv1 = nn.Conv2d(14, 6, 3)  # 6 filters, 3x3 kernel
        self.pool = nn.MaxPool2d(2, 2)    # Max pooling with 2x2 window
        self.conv2 = nn.Conv2d(6, 16, 3)  # 16 filters, 3x3 kernel
        
        # If starting with 8x8, after two pool layers it becomes 2x2.
        # Output from conv2 will be (16 channels, 2x2 feature maps), flattened to 16 * 2 * 2 = 64
        self.fc1 = nn.Linear(16 * 2 * 2, 120)
        self.fc2 = nn.Linear(120, 84)
        # Predicts the tile to move the piece from (64 possible tiles on the board)
        self.fc3 = nn.Linear(84, 64)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply first conv + pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply second conv + pooling
        x = torch.flatten(x, BATCH_SIZE)  # Flatten all dimensions except batch size
        x = F.relu(self.fc1(x))  # Fully connected layer 1
        x = F.relu(self.fc2(x))  # Fully connected layer 2
        x = self.fc3(x)          # Output layer (no activation, logits for classification)
        return x

piece_to_move_net = PieceToMoveNet()
# Move the network to gpu / cpu befor initializing the optimizer
piece_to_move_net.to(device)

optimizer = optim.Adam(piece_to_move_net.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()


**TRAINING LOOP**

In [ ]:
def generate_mask(board: chess.Board, outputs: np.arry, labels: np.array) -> np.array:
    """Creates mask with legal moves from the current board state"""

    mask = np.zeros((8,8)) # 8x8 mask for the chessboard

    # Obtaining legal moves from the board
    legal_moves = list(board.legal_moves)

    # Indicating with 1s the valid squares
    for move in legal_moves:
        to_square = move.to_square
        to_row, to_col = divmod(to_square, 8)
        mask[to_row, to_col] = 1 # A valid square

    # Reshaping mask to match output and labels
    move_mask = mask.flatten() # Converts 8*8 2D array to a 1D array with 64 elements

    masked_outputs = outputs * move_mask
    masked_labels = labels * move_mask

    return masked_outputs, masked_labels



EPOCHS = 10

train_data, validation_data = torch.utils.data.random_split(train_data, [1-TEST_PERCENT, TEST_PERCENT])

training_loader = torch.utils.data.dataLoader(train_data, BATCH_SIZE, shuffle=True, pin_memory=True)
validation_loader = torch.utils.data.dataLoader(validation_data, BATCH_SIZE, shuffle=True, pin_memory=True)




def train_one_epoch(epoch_index: int, tb_writer): 
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        inputs = data[0]
        labels = data[1]

        optimizer.zero_grad()
        outputs = piece_to_move_net(inputs)

        masked_outputs, masked_labels =        
        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999: 
            last_loss = running_loss / 1000
            print(f" batch {i + 1}, loss: {last_loss}")
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar("Loss/train", last_loss, tb_x)
            running_loss = 0.

    return last_loss


for epoch in range(EPOCHS): 
    print(f"EPOCH {epoch + 1}: ")

    model.train(True)
    avg_loss = train_one_epoch(epoch, writer)

    model.train(False)
    running_vloss = 0.0
    for i, v_data in enumerate(validation_loader):
        vinputs = v_data[0]
        vlabels = v_data[1]

        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print(f"LOSS train {avg_loss}, valid {avg_vloss}")

    writer.add_scalars("Training vs Validation Loss", {
        "Training": avg_loss, "Validation": avg_vloss}, 
        epoch + 1)
    writer.flush()
    
    if avg_vloss < best_vloss:
        best_vloss = avg_loss
        model_path = f"model_{timestamp}_{epoch}"
        torch.save(model.state_dict(), model_path)

    epoch += 1